In [ ]:
!pip install pad_sequences
!pip install keras
!pip install tensorflow
!pip install opendatasets

In [ ]:
!pip install keras_preprocessing

In [ ]:
# !unrar x /content/enronsent.tar.gz
!unzip "/content/enronsentv.zip" -d "/content"

Archive:  /content/enronsentv.zip
   creating: /content/enronsentv/
  inflating: /content/enronsentv/enronsent00  
  inflating: /content/enronsentv/enronsent01  
  inflating: /content/enronsentv/enronsent02  
  inflating: /content/enronsentv/enronsent03  
  inflating: /content/enronsentv/enronsent04  
  inflating: /content/enronsentv/enronsent05  
  inflating: /content/enronsentv/enronsent06  
  inflating: /content/enronsentv/enronsent07  
  inflating: /content/enronsentv/enronsent08  
  inflating: /content/enronsentv/enronsent09  
  inflating: /content/enronsentv/enronsent10  
  inflating: /content/enronsentv/enronsent11  
  inflating: /content/enronsentv/enronsent12  
  inflating: /content/enronsentv/enronsent13  
  inflating: /content/enronsentv/enronsent14  
  inflating: /content/enronsentv/enronsent15  


In [ ]:
# import liberaris
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.layers import Dense
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.utils import *
import os
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from gensim.models import Word2Vec
from nltk.tokenize import sent_tokenize
from nltk.corpus import PlaintextCorpusReader
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Download the necessary data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Set the path to the Enron Sent Corpus Dataset
corpus_root = '/content/enronsentv'

# Set the file ids of the data files
file_ids_train = ['enronsent00', 'enronsent01','enronsent02', 'enronsent03', 'enronsent04',
            'enronsent05','enronsent06','enronsent07','enronsent08','enronsent09','enronsent10']
file_ids_test = ['enronsent11','enronsent12','enronsent13','enronsent14', 'enronsent15']

# Initialize an empty list to store the tokens
tokens_train = []
tokens_test = []

# Load and process each data file
for file_id in file_ids_train:
    # Load the data
    corpus = PlaintextCorpusReader(corpus_root, file_id)
    # Tokenize the data
    tokens_train.extend(corpus.sents())

# Load and process each data file
for file_id in file_ids_test:
    # Load the data
    corpus = PlaintextCorpusReader(corpus_root, file_id)
    # Tokenize the data
    tokens_test.extend(corpus.sents())

In [ ]:
# Remove punctuation, numbers, and lemmatize the tokens
# we will not remove stopwords as we also need to predict it
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

# Preprocess the training data
tokens_train = [[lemmatizer.lemmatize(token) for token in sent if token not in punctuation and not token.isdigit()] for sent in tokens_train]

# Preprocess the testing data
tokens_test = [[lemmatizer.lemmatize(token) for token in sent if token not in punctuation and not token.isdigit()] for sent in tokens_test]

In [ ]:
# Set the number of time steps
time_steps = 5
x_train = []
y_train = []
x_test = []
y_test = []

# Split each sample into fixed time steps for training data
for token in tokens_train:
    # Create input and output sequences
    for i in range(len(token) - time_steps):
        # Extract the input sequence of length time_steps
        x_train.append(token[i:i+time_steps])
        # Extract the next word as the output sequence
        y_train.append(token[i+time_steps])

# Split each sample into fixed time steps for testing data
for token in tokens_test:
    # Create input and output sequences
    for i in range(len(token) - time_steps):
        # Extract the input sequence of length time_steps
        x_test.append(token[i:i+time_steps])
        # Extract the next word as the output sequence
        y_test.append(token[i+time_steps])

In [ ]:
# Train our word2vec model
# Combine training and testing tokens into a single list
word_train = tokens_train + tokens_test

# Train Word2Vec model on the combined tokens
word_model = Word2Vec(word_train, min_count=1, vector_size=25)

# Save the trained Word2Vec model
word_model.save("word2vec_model.bin")

In [ ]:
# get the vocabulary size and embedding dimension from the pre-trained model
vocab_size = len(word_model.wv.index_to_key)
embedding_dim = word_model.wv.vector_size

# create a weight matrix for the embedding layer
embedding_matrix = word_model.wv.vectors

# print the info
print(f"""vocab size : {vocab_size}
embedding_dim: {embedding_dim}
embedding_matrix shape: {embedding_matrix.shape}""")

vocab size : 87090
embedding_dim: 25
embedding_matrix shape: (87090, 25)


In [ ]:
# create a tokenizer and fit it on our text data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)

# encode the text data as sequences of integers
x_train_vectors = tokenizer.texts_to_sequences(x_train)
x_test_vectors = tokenizer.texts_to_sequences(x_test)

# pad the sequences to have the same length
maxlen = max(len(seq) for seq in x_train)
x_train_vectors = pad_sequences(x_train_vectors, maxlen=time_steps)
x_test_vectors = pad_sequences(x_test_vectors, maxlen=time_steps)

In [ ]:
# create a tokenizer and fit it on our text data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(y_train)

# encode the text data as sequences of integers
y_train_vectors = tokenizer.texts_to_sequences(y_train)
y_test_vectors = tokenizer.texts_to_sequences(y_test)

# pad the sequences to have the same length
maxlen = max(len(seq) for seq in x_train)
y_train_vectors = pad_sequences(y_train_vectors, maxlen=1)
y_test_vectors = pad_sequences(y_test_vectors, maxlen=1)

In [ ]:
# Converting the data to numpy arrays
x_train_vectors = np.array(x_train_vectors)
y_train_vectors = np.array(y_train_vectors)
x_test_vectors = np.array(x_test_vectors)
y_test_vectors = np.array(y_test_vectors)

In [ ]:
x_train_vectors.shape

(2663198, 5)

In [ ]:
# create the LSTM model
model = Sequential()

# Add an embedding layer with pre-trained word embeddings
model.add(Embedding(vocab_size, embedding_dim, weights=[embedding_matrix], input_length=time_steps, trainable=False))

# Add batch normalization layer
model.add(BatchNormalization())

# Add LSTM layer
model.add(LSTM(128))

# Add a dense layer with activation function 'relu'
model.add(Dense(1, activation='relu'))

# Compile the model with mean squared error loss and Adam optimizer
model.compile(loss='mse', optimizer='adam')

In [ ]:
# train the LSTM model on your data
model.fit(x_train_vectors,y_train_vectors, validation_data=(x_test_vectors, y_test_vectors), epochs=10, verbose=1,batch_size=64)

Epoch 1/10
41613/41613 [==============================] - 243s 6ms/step - loss: 17607494.0000 - val_loss: 15952373.0000
Epoch 2/10
41613/41613 [==============================] - 230s 6ms/step - loss: 16973204.0000 - val_loss: 15820531.0000
Epoch 3/10
41613/41613 [==============================] - 232s 6ms/step - loss: 16767962.0000 - val_loss: 15790979.0000
Epoch 4/10
41613/41613 [==============================] - 218s 5ms/step - loss: 16635647.0000 - val_loss: 15793943.0000
Epoch 5/10
41613/41613 [==============================] - 228s 5ms/step - loss: 16541855.0000 - val_loss: 15814003.0000
Epoch 6/10
41613/41613 [==============================] - 227s 5ms/step - loss: 16466976.0000 - val_loss: 15782689.0000
Epoch 7/10
41613/41613 [==============================] - 223s 5ms/step - loss: 16408297.0000 - val_loss: 15753175.0000
Epoch 8/10
41613/41613 [==============================] - 217s 5ms/step - loss: 16354979.0000 - val_loss: 15782681.0000
Epoch 9/10
41613/41613 [================

<h3>Note that it is difficult to use vectors and achieve excellent results due to poor computational power that we have and the ram of google colab was crashing so we convert the problem into a linear problem

In [ ]:
def prepare_input():
    # Get user input
    input_str = input()

    # Check if user wants to exit
    if input_str == '-1':
        return -1, ''

    # Tokenize the input sentence
    tokenized_word = tokenizer.texts_to_sequences([input_str])

    # Pad the tokenized input sequence to have length 1
    padded_word = pad_sequences(tokenized_word, maxlen=1)

    return padded_word, input_str

In [ ]:
def prepare_output(number):
    # Pad the number sequence to have length 1
    padded_sequence = pad_sequences(number, maxlen=1)

    # Convert the padded sequence back to text
    text = tokenizer.sequences_to_texts(padded_sequence)

    # Return the predicted word as text
    return text[0]

In [ ]:
def test():
  first_loop = True
  correct = False
  sentence = ""

  while True:
      if first_loop:
          print("Enter Next word (-1 to terminate):")
          first_loop = False
      else:
          if not correct:
              print("Sorry, Enter Next word (-1 to terminate):")

      if correct:
          # Predict the next word based on the previous prediction
          tokenized_word = tokenizer.texts_to_sequences([predicted_word])
          predicted_word = pad_sequences(tokenized_word, maxlen=1)
          predicted_word = model.predict(predicted_word, verbose=0)
          predicted_word = prepare_output(predicted_word)
      else:
          # Get the input word from the user
          word, string = prepare_input()
          if word == -1:
              print(f"Your final Sentence is “{sentence}”")
              break
          sentence += string + " "

          # Predict the next word based on the user input
          predicted_word = model.predict(word, verbose=0)
          predicted_word = prepare_output(predicted_word)

      print(f"Is your next word: “{predicted_word}”? [y] = yes , [n] = no")
      ans = input()
      if ans == 'y':
          correct = True
          sentence += predicted_word + " "
          continue
      else:
          correct = False
          continue

In [ ]:
test()

Enter Next word (-1 to terminate):
i
Is your next word: “shall”? [y] = yes , [n] = no
y
Is your next word: “suggested”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
go
Is your next word: “significant”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
to
Is your next word: “nice”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
the
Is your next word: “agent”? [y] = yes , [n] = no
y
Is your next word: “therefore”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
and
Is your next word: “hit”? [y] = yes , [n] = no
y
Is your next word: “hearing”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
him
Is your next word: “executive”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
-1
Your final Sentence is “i shall go to the agent and hit him ”


In [ ]:
test()

Enter Next word (-1 to terminate):
my
Is your next word: “supplier”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
name
Is your next word: “need”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
is
Is your next word: “owned”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
marwan
Is your next word: “weekly”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
-1
Your final Sentence is “my name is marwan ”


In [ ]:
test()

Enter Next word (-1 to terminate):
do
Is your next word: “organization”? [y] = yes , [n] = no
y
Is your next word: “speaker”? [y] = yes , [n] = no
y
Is your next word: “central”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
is
Is your next word: “owned”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
a
Is your next word: “pressure”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
loyer
Is your next word: “weekly”? [y] = yes , [n] = no
n
Sorry, Enter Next word (-1 to terminate):
-1
Your final Sentence is “do organization speaker is a loyer ”
